In [4]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import os
import pickle as pk
import numpy as np

CIFAR_DIR ='../../datasets/cifar-10-batches-py'
dirs = os.listdir( CIFAR_DIR )
print(dirs)

Instructions for updating:
non-resource variables are not supported in the long term
['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [14]:
def load_data(filename):
    """read data from data file"""
    with open(filename,'rb') as f:
        data = pk.load(f,encoding='iso-8859-1')
        return (data['data'],data['labels'])

class CifarData:
    def __init__(self,filenames,need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data ,labels = load_data(filename)
            for item , label in zip(data,labels):
                if label in [0,1]:
                    all_data.append(item)
                    all_labels.append(label)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 -1  #归一化
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        """打乱数据集"""
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._lable = self._labels[p]
        
    def next_batch(self,batch_size):
        """ 随机取batch_size个训练样本  """
        end_indicator = self._indicator + batch_size
        if end_indicator < self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = self._indicator + batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch_size is larger than examples")
            
        batch_data = self._data[self._indicator:end_indicator]
        batch_lables = self._labels[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data , batch_lables


train_filenames = [os.path.join(CIFAR_DIR,'data_batch_%d' %i) for i in range (1,6)]
test_filenames = [os.path.join(CIFAR_DIR,'test_batch')]

train_data = CifarData(train_filenames,True)
tets_data = CifarData(test_filenames,True)
#batch_data , batch_lables = train_data.next_batch(10)

"""
print(batch_data)
print(batch_lables)
"""

(10000, 3072)
(10000,)
(2000, 3072)
(2000,)


'\nprint(batch_data)\nprint(batch_lables)\n'

In [15]:

tf.reset_default_graph()

x = tf.placeholder(tf.float32,[None,3072])  #定义一个空的数组
y = tf.placeholder(tf.int64,[None])

#w:[3072*1]
w = tf.get_variable('w',[x.get_shape()[-1],1],initializer=tf.random_normal_initializer(0,1))
#[1,]
b = tf.get_variable('b',[1], initializer=tf.constant_initializer(0.0))

#Y_shape:[None,3072]*[3072*1]+{1,} = [None,1]+[1,]
y_ = tf.matmul(x,w) + b #matmul做矩阵乘法 matrix multiply == np.dot()

#p_y_1.shape = [None,1],
p_y_1 = tf.nn.sigmoid(y_) #y=1的概率值

#因为y.shape = [None,], p_y_1.shape = [None,1],所以需要转换成相同的类型
#reshape_y : [None,1]
reshape_y = tf.reshape(y,(-1,1))
#cast 类型变换
reshape_y_float = tf.cast(reshape_y,tf.float32)
#reduce_mean求均值
loss = tf.reduce_mean(tf.square(reshape_y_float - p_y_1 )) 

predict  = p_y_1>0.5

correct_predict = tf.equal(tf.cast(predict,tf.int64),reshape_y)

accuracy = tf.reduce_mean(tf.cast(correct_predict,tf.float32))

#定义一块名为train_op的区域，并在其中工作
with tf.name_scope('train_op'): 
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss) #AdamOptimizer优化器的学习率是1e-3,来最小化loss函数
    

In [20]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data , batch_labels = train_data.next_batch(batch_size)
        loss_val,acc_val,_=sess.run(
            [loss,accuracy,train_op],
            feed_dict = {x:batch_data,
                         y:batch_labels})
        if (i+1) % 500  == 0:
            print('[Train] Step: %d,Loss:%4.5f,acc:%4.5f'
                  %(i+1,loss_val,acc_val))
        
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames,False)
            all_test_acc = []
            for j in range(test_steps):
                
                test_batch_data , test_batch_labels  = test.next_batch(batch_size)
                test_acc_val = sess.run([accurary],feed_dict={x:test_batch_data, y:test_batch_labels})
                all_test_acc.append(test_acc_val)
            
            test_acc = np.mean(all_test_acc)
            print("[Test] Step: %d , Acc: %4.5f" % (i+1,test_acc))
                
            
       
        

[Train Step]: 500,Loss:0.15000,acc:0.85000
[Train Step]: 1000,Loss:0.35000,acc:0.65000
[Train Step]: 1500,Loss:0.30000,acc:0.70000
[Train Step]: 2000,Loss:0.30000,acc:0.70000
[Train Step]: 2500,Loss:0.30000,acc:0.70000
[Train Step]: 3000,Loss:0.35000,acc:0.65000
[Train Step]: 3500,Loss:0.30000,acc:0.70000
[Train Step]: 4000,Loss:0.30000,acc:0.70000
[Train Step]: 4500,Loss:0.30000,acc:0.70000
[Train Step]: 5000,Loss:0.35000,acc:0.65000
[Train Step]: 5500,Loss:0.35000,acc:0.65000
[Train Step]: 6000,Loss:0.30000,acc:0.70000
[Train Step]: 6500,Loss:0.30000,acc:0.70000
[Train Step]: 7000,Loss:0.30000,acc:0.70000
[Train Step]: 7500,Loss:0.30000,acc:0.70000
[Train Step]: 8000,Loss:0.30000,acc:0.70000
[Train Step]: 8500,Loss:0.30000,acc:0.70000
[Train Step]: 9000,Loss:0.30000,acc:0.70000
[Train Step]: 9500,Loss:0.30000,acc:0.70000
[Train Step]: 10000,Loss:0.30000,acc:0.70000
